<a href="https://colab.research.google.com/github/Mzluci9/Complaint-Analysis-RAG/blob/task_1/notebooks/task4_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Task 4: RAG Chatbot Evaluation and Optimization
**Participant**: Michael Zewdu Lemma
**Cohort**: KAIM 5/6
**Date**: July 7, 2025

This notebook evaluates the RAG chatbot from Task 3, assesses retrieval and generation performance, and optimizes the pipeline for better accuracy and efficiency.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load Task 3 results
results_path = '/content/drive/My Drive/Colab Notebooks/rag_results.json'
try:
    results = pd.read_json(results_path, orient='records', lines=True)
    logging.info(f"Loaded results with shape: {results.shape}")
    print("Sample Results:")
    print(results[['query', 'response']].head())
except Exception as e:
    logging.error(f"Failed to load results: {e}")
    raise

# Evaluate Baseline Performance

In [ ]:
# Manual evaluation of response quality
for idx, row in results.iterrows():
    print(f"\nQuery: {row['query']}")
    print(f"Response: {row['response']}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(row['retrieved_chunks']):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {row['distances'][i]:.4f}):")
        print(chunk['chunk_text'])

In [ ]:
!pip install -U bitsandbytes
!pip install -U accelerate transformers


# Test Additional Queries

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Load data and index
data_path = '/content/drive/My Drive/Colab Notebooks/chunked_complaints.csv'
index_path = '//content/drive/My Drive/Colab Notebooks/complaint_index.faiss'
df_chunks = pd.read_csv(data_path)
index = faiss.read_index(index_path)

# Load LLM
model_name = "HuggingFaceH4/zephyr-7b-beta"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Retriever function
embedder = SentenceTransformer('all-MiniLM-L6-v2')
def retrieve_chunks(query, index, df_chunks, embedder, top_k=5):
    query_embedding = embedder.encode([query], show_progress_bar=False)
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    retrieved_chunks = df_chunks.iloc[indices[0]][['complaint_id', 'product', 'chunk_idx', 'chunk_text']].to_dict('records')
    return retrieved_chunks, distances[0]

# RAG pipeline
def rag_pipeline(query, index, df_chunks, embedder, llm, top_k=5):
    chunks, distances = retrieve_chunks(query, index, df_chunks, embedder, top_k)
    if not chunks:
        return "No relevant complaints found.", [], []
    system_prompt = "You are a financial complaint analysis assistant."
    context = "\n".join([f"Complaint (Product: {chunk['product']}): {chunk['chunk_text']}" for chunk in chunks])
    full_prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{query}\n<|retrieved|>\n{context}"
    response = llm(full_prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
    return response[0]["generated_text"], chunks, distances

# Test new queries
new_queries = [
    "What are the main reasons for Credit Card dissatisfaction?",
    "Why do people complain about Personal Loan interest rates?",
    "What problems occur with Money Transfer delays?"
]
new_results = []
for query in new_queries:
    response, chunks, distances = rag_pipeline(query, index, df_chunks, embedder, llm)
    print(f"\nQuery: {query}")
    print(f"Response: {response}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {distances[i]:.4f}):")
        print(chunk['chunk_text'])
    new_results.append({
        'query': query,
        'response': response,
        'retrieved_chunks': chunks,
        'distances': distances.tolist()
    })

# Optimize the Pipeline

In [ ]:
# Optimization 1: Adjust top_k
print("\nTesting top_k=3")
for query in new_queries[:2]:  # Test on 2 queries
    response, chunks, distances = rag_pipeline(query, index, df_chunks, embedder, llm, top_k=3)
    print(f"\nQuery: {query}")
    print(f"Response (top_k=3): {response}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {distances[i]:.4f}):")
        print(chunk['chunk_text'])

# Optimization 2: Filter short chunks (<5 words)
df_filtered = df_chunks[df_chunks['chunk_length'] >= 5]
print("\nFiltered Chunked Dataset Shape:", df_filtered.shape)
for query in new_queries[:2]:
    response, chunks, distances = rag_pipeline(query, index, df_filtered, embedder, llm)
    print(f"\nQuery (Filtered): {query}")
    print(f"Response: {response}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {distances[i]:.4f}):")
        print(chunk['chunk_text'])

# Optimization 3: Tweak prompt
def rag_pipeline_optimized(query, index, df_chunks, embedder, llm, top_k=5):
    chunks, distances = retrieve_chunks(query, index, df_chunks, embedder, top_k)
    if not chunks:
        return "No relevant complaints found.", [], []
    system_prompt = "You are a financial complaint analysis assistant. Provide a concise, bullet-point summary of the main issues based on the complaints."
    context = "\n".join([f"Complaint (Product: {chunk['product']}): {chunk['chunk_text']}" for chunk in chunks])
    full_prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{query}\n<|retrieved|>\n{context}"
    response = llm(full_prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
    return response[0]["generated_text"], chunks, distances

print("\nTesting optimized prompt")
for query in new_queries[:2]:
    response, chunks, distances = rag_pipeline_optimized(query, index, df_chunks, embedder, llm)
    print(f"\nQuery (Optimized Prompt): {query}")
    print(f"Response: {response}")
    print("Retrieved Chunks:")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1} (Product: {chunk['product']}, Distance: {distances[i]:.4f}):")
        print(chunk['chunk_text'])

# Save Results and Update Report

In [8]:
from google.colab import files

# Save new results
all_results = results.to_dict('records') + new_results
pd.DataFrame(all_results).to_json('/content/drive/My Drive/Colab Notebooks/rag_results_updated.json', orient='records', lines=True)
files.download('/content/drive/My Drive/Colab Notebooks/rag_results_updated.json')

# Save notebook
with open('/content/drive/My Drive/Colab Notebooks/Task4_Evaluation.ipynb', 'w') as f:
    f.write("# [Manually save notebook content]")
files.download('/content/drive/My Drive/Colab Notebooks/Task4_Evaluation.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>